In [1]:
from guidance import models, gen, select
import guidance
lm = models.Transformers('TheBloke/Tess-10.7B-v1.5b-GPTQ', device_map="cuda", echo=True, temperature=50, max_length = 10)


In [24]:
from datetime import date

conversation = []

def get_conversation(conversation=conversation, retrieval_amount=8):
    if len(conversation) == 0:
        return 'No Record Yet.'
    else:
        prompt = ''
        if len(conversation) < retrieval_amount:
            for i in range(len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        else:
            start_index = max(len(conversation) - retrieval_amount, 0)
            for i in range(start_index, len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        return prompt.strip()

# Example usage:
conversation = [
    {'source': 'user', 'content': 'Hello!', 'timestamp': 1647289200},
    {'source': 'assistant', 'content': 'Hi there!', 'timestamp': 1647292800},
    {'source': 'user', 'content': 'How are you?', 'timestamp': 1647296400},
    {'source': 'assistant', 'content': "I'm doing well, thank you!", 'timestamp': 1647300000},
]

print(get_conversation(conversation))


[2022-03-15] user: Hello!
[2022-03-15] assistant: Hi there!
[2022-03-15] user: How are you?
[2022-03-15] assistant: I'm doing well, thank you!


In [2]:
def yield_test(n):
    print("start n =", n)
    for i in range(n):
        yield i*i
        print("i =", i)
        if i == 2:  # Add a condition to return early
            return
    print("end")

tests = yield_test(5)
for test in tests:
    print("test =", test)
    print("--------")

start n = 5
test = 0
--------
i = 0
test = 1
--------
i = 1
test = 4
--------
i = 2


In [14]:
lm2 =lm+ "Hello! I "+gen(max_tokens=10, temperature=0.35)

In [7]:

import chromadb
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import time
from datetime import date

c:\Users\User\miniconda3\envs\Lucid\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:

#embeddings = sentence_transformer.encode(sentences)#
short_term_memory = chromadb.Client()
#client = chromadb.PersistentClient(path="./Chroma")
working_memory = []
"""
This is what a Info Block should look like
{
	'object_type' : 'entity',
	'object_name' : 'Miko',
	'content' : 'Miko like Nintendo games.',
	'timestamp' : '2021-10-15 17:37:00',
	'vector' array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,]], dtype=float32): ,
}    

with guidance
f'''\
{
	'object_type' : '{select(['entity','events'])'},
	'object_name' : '{gen(stop='\'')},
	'content' : '{gen(stop='\'')},
	'timestamp' : {date.fromtimestamp(time.time())},
	'vector' : {sentence_transformer.encode([])},
}'''
"""
#def process_new_info_block(lm_text_result):

@guidance(stateless=True)
def guidance_make_new_info_block(lm, passage):
		first_curly = "{"
		second_curly = "}"
		lm += """\
This is what a Info Block should look like from an example:
[Example]
Lucid: What are you doing now? Stop ignoring me!
Miko: I'm playing Nintendo games.
Lucid: And that's a higher priority than me?
Miko: ...
Lucid: Your silence speaks volumes.

```json
{
	"object_type" : 'entity',
	"object_name" : 'Miko',
	"content" : 'Miko likes Nintendo games.',
}
```
[End of Example]
"""
		lm += f"""\
{passage}
```json
{first_curly}
	"object_type" : '{select(['entity','events'], name="object_type")},
	"object_name" : "{gen(stop='"',name = "object_name")},
	"content" : "{gen(stop='"',name="content")},"""  
		return lm

def make_new_info_block(lm, passage):
	first_curly = "{"
	second_curly = "}"
	lm += guidance_make_new_info_block(passage)
	print(lm['object_type'],lm['object_name'],lm['content'])
	timestamp = date.fromtimestamp(time.time())
	json_file ={
	"object_type" : lm['object_type'],
	"object_name" : lm['object_name'],
	"content" : lm['content'],
	"timestamp" : timestamp,
	"vector" : sentence_transformer.encode([lm['content']]),
	}
	return json_file

In [9]:
passage = """\
Lucid: Hey Miko what's your favorite food?
Miko: Umm I don't know maybe rice?
Lucid you are really boring you know?"""
make_new_info_block(lm, passage)

entity Miko Miko likes rice.


{'object_type': 'entity',
 'object_name': 'Miko',
 'content': 'Miko likes rice.',
 'timestamp': datetime.date(2024, 3, 11),
 'vector': array([[-3.82033326e-02, -4.41594794e-02,  2.91311648e-02,
          5.75081445e-02,  1.31961489e-02, -4.42584679e-02,
          1.10918455e-01, -1.14079034e-02, -2.05194596e-02,
         -7.49267032e-03,  1.11112319e-01, -5.89605682e-02,
         -7.79123306e-02,  4.89122346e-02, -3.07194103e-04,
          4.11266088e-02,  7.78212994e-02,  1.50255887e-02,
         -5.94113693e-02, -4.20605876e-02, -1.76906902e-02,
         -5.96213378e-02,  5.38771525e-02, -1.14128571e-02,
          2.50688680e-02, -2.08598375e-02,  4.13506739e-02,
          8.09722990e-02,  1.53833143e-02, -4.60688397e-03,
         -3.18389349e-02,  9.99455601e-02,  1.68346763e-02,
         -8.45997594e-03,  1.17826229e-03,  9.32732373e-02,
          7.22661912e-02, -9.07012001e-02,  1.61419287e-02,
         -4.43259068e-02, -3.78268622e-02, -7.43800625e-02,
          7.32221901e-02, 

In [ ]:
import sys
import traceback


class Item:
    SPACE = ' '
    DASH_SPACE = ' - '
    def __init__(self, key, name, function, parent=None):
        self.key = str(key)
        self.name = name
        self.function = function
        self.parent = parent

        if parent:
            parent.add_item(self)

    def __str__(self):
        return self.SPACE + self.key + self.DASH_SPACE + self.name

    def execute(self):
        print('\n' + self.name + '\n')
        return self.function()


class Menu:
    def __init__(self, name, items=None):
        self.name = name
        self.items = items or []

    def add_item(self, item):
        self.items.append(item)
        if item.parent != self:
            item.parent = self

    def remove_item(self, item):
        self.items.remove(item)
        if item.parent == self:
            item.parent = None

    def __str__(self):
        all = ['', self.name, '']
        for item in self.items:
            all.append(str(item))
        return '\n'.join(all)
            
    def get_item(self, key):
        for item in self.items:
            if item.key == key:
                return item

    def execute(self):
        while True:
            try:
                print(self)
                choice = raw_input(" >>  ")
                try:
                    item = self.get_item(choice)

                    try:
                        item.execute()
                    except Exception:
                        traceback.print_exc()

                except Exception:
                    print('Invalid selection, please try again.\n')
            except KeyboardInterrupt:
                sys.exit()
    
    
if __name__ == "__main__":
    # Pass in methods to run for each menu item
    menu_items = [
        Item(1, 'View Running tasks', view_running_tasks),
        Item(2, 'View Task', view_task),
        # ...
        Item(9, 'Exit', exit)]
    menu = Menu('Task Management', items=menu_items)

    menu.execute()